##### Импорты

In [219]:
import random
import math


### Одинарная перестановка

In [220]:
def singlePermutationCipher(text, order, decrypt=False):
    if decrypt:
        return ''.join([a[0] for a in sorted(zip(text, order),
                                             key=lambda tup: tup[1])])
    return ''.join([text[i] for i in order])


def getPermutationsOrder(length, seed):
    random.seed(seed)
    result = [i for i in range(0, length, 1)]
    random.shuffle(result)
    return result


text = 'Isyuk'
order = getPermutationsOrder(len(text), 18)
encrypt = singlePermutationCipher(text, order)
decrypt = singlePermutationCipher(encrypt, order, decrypt=True)
print(f'''
Порядок перестановки: {order}   
Исходный текст:         {text}
Зашифрованный текст:    {encrypt}
Дешифрованный текст:    {decrypt}
''')


Порядок перестановки: [3, 4, 2, 0, 1]   
Исходный текст:         Isyuk
Зашифрованный текст:    ukyIs
Дешифрованный текст:    Isyuk



### Одинарная блочная перестановка

In [221]:

def singleBlockPermutationCipher(text, order, decrypt=False):
    result = ''
    if len(text) % len(order) != 0:
        text += 'x' * (len(order) - len(text) % len(order))

    for i in range(0, len(text), len(order)):
        chunk = text[i:i+len(order)]
        if decrypt:
            result += ''.join([a[0] for a in sorted(zip(chunk,
                              order), key=lambda tup: tup[1])])
            continue
        for j in order:
            result += chunk[j]
    return result

text = 'Isyuk'
order = getPermutationsOrder(7, 9)
encrypt = singleBlockPermutationCipher(text, order)
decrypt = singleBlockPermutationCipher(encrypt, order, decrypt=True)
print(f'''
Порядок перестановки: {order}   
Исходный текст:         {text}
Зашифрованный текст:    {encrypt}
Дешифрованный текст:    {decrypt}
''')


Порядок перестановки: [1, 6, 0, 5, 2, 4, 3]   
Исходный текст:         Isyuk
Зашифрованный текст:    sxIxyku
Дешифрованный текст:    Isyukxx



### Табличная маршрутная перестановка

In [222]:
def routeCipher(text, grid, decrypt=False):
    if decrypt:
        return routeCipherDecrypt(text, len(grid), len(grid[0]))
    return routeCipherEncrypt(text, grid)


def routeCipherDecrypt(text, rows, cols):
    grid = spiralGrid(text, rows, cols)
    return ''.join([char for row in grid for char in row])


def routeCipherEncrypt(text, grid):
    result = ''
    x = 0
    y = 0
    cols = len(grid[x])
    rows = len(grid)

    while (x < rows and y < cols):
        result += ''.join(grid[x][y:cols])
        x += 1
        for i in range(x, rows, 1):
            result += grid[i][cols - 1]
        cols -= 1
        result += ''.join(reversed(grid[rows - 1][y:cols]))
        rows -= 1
        for i in range(rows - 1, x - 1, -1):
            result += grid[i][y]
        y += 1

    return result


def spiralGrid(text, rows, cols):
    grid = [['x' for j in range(cols)] for i in range(rows)]
    x = 0
    y = 0
    dx = 0
    dy = 1
    for i in range(1, rows*cols+1):
        grid[x][y] = text[i - 1]
        if x + dx < 0 or x + dx >= rows or y + dy < 0 or y + dy >= cols or grid[x+dx][y+dy] != 'x':
            dx, dy = dy, -dx
        x += dx
        y += dy
    return grid


def getGrid(text, rows, cols):
    if len(text) < rows * cols:
        text += 'x' * (rows * cols - len(text))
    grid = []
    for i in range(rows):
        grid.append([])
        for j in range(cols):
            grid[i].append(text[i * cols + j])
    return grid


text = 'Isyuk Nikita'
grid = getGrid(text, 5, 3)
encrypt = routeCipher(text, grid)
decrypt = routeCipher(encrypt, grid, decrypt=True)
print(
    'Таблица: (записана слева-направо сверху-вниз\n' +
    '\t  шифруется по спирали из верхнего левого угла)\n' +
    '{}'.format('\n'.join([str(row) for row in grid]) +
                '\nИсходный текст:         {}\n'.format(text) +
                'Зашифрованный текст:    {}\n'.format(encrypt) +
                'Дешифрованный текст:    {}'.format(decrypt)))

Таблица: (записана слева-направо сверху-вниз
	  шифруется по спирали из верхнего левого угла)
['I', 's', 'y']
['u', 'k', ' ']
['N', 'i', 'k']
['i', 't', 'a']
['x', 'x', 'x']
Исходный текст:         Isyuk Nikita
Зашифрованный текст:    Isy kaxxxiNukiti
Дешифрованный текст:    Isyuk Nikitaxxx


### Вертикальная перестановка

In [223]:
def verticalPermutationCipher(text, key, decrypt=False):
    if decrypt:
        return verticalPermutationCipherDecrypt(text, key)
    return verticalPermutationCipherEncrypt(text, key)


def verticalPermutationCipherDecrypt(text, key):
    result = ''
    keyValues = getKeyValues(key)
    indices = convertToIndices(keyValues)
    grid = getGrid(text, rows=len(indices))
    sortedRotatedMatrix = [grid[i] for i in indices]
    rotatedMatrix = rotateMatrixClockwise90(sortedRotatedMatrix)
    result += ''.join([''.join([char for char in row])
                      for row in rotatedMatrix])
    return result


def verticalPermutationCipherEncrypt(text, key):
    result = ''
    keyValues = getKeyValues(key)
    indices = convertToIndices(keyValues)
    grid = getGrid(text, cols=len(indices))
    rotatedMatrix = rotateMatrixAntiClockwise90(grid)
    sortedRotatedMatrix = [rotatedMatrix[i] for i in indices]
    sortedMatrix = rotateMatrixClockwise90(sortedRotatedMatrix)
    sortedMatrix = rotateMatrixAntiClockwise90(sortedMatrix)
    result += ''.join([''.join([char for char in row])
                      for row in sortedMatrix])
    return result


def convertToIndices(list):
    sorted_indices = sorted(range(len(list)), key=lambda k: list[k])
    indexes = [0]*len(list)
    for i, x in enumerate(sorted_indices):
        indexes[x] = i
    return indexes


def getKeyValues(key):
    numbers = []
    for char in key:
        if not char.isalpha():
            return 'key can contain only alpha characters'
        isLower = char.islower()
        numbers.append(ord(char) - ord('a' if isLower else 'A') + 1)

    seen = dict(zip(numbers, [0 for i in range(len(numbers))]))
    keyValues = []
    for num in numbers:
        keyValues.append(num + seen[num])
        seen[num] += 1
    return keyValues


def rotateMatrixAntiClockwise90(matrix):
    return [list(col) for col in reversed(list(zip(*matrix)))]


def rotateMatrixClockwise90(matrix):
    return list(map(list, zip(*matrix)))


def getGrid(text, cols=0, rows=0):
    grid = []
    if rows == 0:
        rows = math.ceil(len(text) / cols)
    if cols == 0:
        cols = math.ceil(len(text) / rows)
    if len(text) < rows * cols:
        text += 'x' * (rows * cols - len(text))
    for i in range(rows):
        grid.append([])
        for j in range(cols):
            grid[i].append(text[i * cols + j])
    return grid


text = 'Isyuk Nikita'
key = 'meh'
keyValues = getKeyValues(key)
encrypt = verticalPermutationCipher(text, key)
decrypt = verticalPermutationCipher(encrypt, key, decrypt=True)
print(
    'Таблица: \n' +
    '{} - (ключ)'.format(list(key)) +
    '\n{}'.format([str(item) for item in convertToIndices(getKeyValues(key))]) +
    '\n{}'.format(''.join(['-' for i in range(len(key) * 5)])) +
    '\n{}'.format('\n'.join([str(row) for row in getGrid(text, cols=len(key))]) +
                  '\nИсходный текст:         {}\n'.format(text) +
                  'Зашифрованный текст:    {}\n'.format(encrypt) +
                  'Дешифрованный текст:    {}'.format(decrypt)))

Таблица: 
['m', 'e', 'h'] - (ключ)
['2', '0', '1']
---------------
['I', 's', 'y']
['u', 'k', ' ']
['N', 'i', 'k']
['i', 't', 'a']
Исходный текст:         Isyuk Nikita
Зашифрованный текст:    skity kaIuNi
Дешифрованный текст:    Isyuk Nikita


### Поворотная решетка

In [224]:
def grilleCipherEncrypt(text, mask):
    result = ''
    size = len(mask)
    if len(text) < size * size:
        text += 'x' * (size * size - len(text))

    resultGrid = [['' for _ in range(size)]for _ in range(size)]

    for _ in range(4):
        for i in range(size):
            for j in range(size):
                if mask[i][j] == 1:
                    resultGrid[i][j] = text[0]
                    text = text[1:]
        mask = rotateMask(mask)

    resultGrid = rotateMatrixClockwise90(resultGrid)
    result += ''.join(''.join([char for char in row]) for row in resultGrid)
    return result


def grilleCipherDecrypt(text, mask):
    result = ''
    size = len(mask)
    resultGrid = [[text[i * size + j]
                   for i in range(size)]for j in range(size)]

    for _ in range(4):
        for i in range(size):
            for j in range(size):
                if mask[i][j] == 1:
                    result += resultGrid[i][j]
        mask = rotateMask(mask)

    return result


def rotateMask(mask):
    size = len(mask)
    new_mask = [[False for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(size):
            new_mask[j][size - i - 1] = mask[i][j]
    return new_mask


text = 'Isyuk Nikita'
mask = [
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
]
encrypt = grilleCipherEncrypt(text, mask)
decrypt = grilleCipherDecrypt(encrypt, mask)
print(
    'Маска: (поворачивается по часовой на 90 градусов 4 раза)\n' +
    '{}'.format('\n'.join([str(row) for row in mask])) +
    '\nТаблица: (порядок выписывания: ↓↗)\n' +
    '{}'.format('\n'.join([str(row) for row in getGrid(text, len(mask), len(mask))]) +
                '\nИсходный текст:         {}\n'.format(text) +
                'Зашифрованный текст:    {}\n'.format(encrypt) +
                'Дешифрованный текст:    {}'.format(decrypt)))

Маска: (поворачивается по часовой на 90 градусов 4 раза)
[1, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 1, 0, 0]
Таблица: (порядок выписывания: ↓↗)
['I', 's', 'y', 'u']
['k', ' ', 'N', 'i']
['k', 'i', 't', 'a']
['x', 'x', 'x', 'x']
Исходный текст:         Isyuk Nikita
Зашифрованный текст:    I txxiNukxyiksxa
Дешифрованный текст:    Isyuk Nikitaxxxx


### Магический квадрат

In [225]:
def magicSquareEncrypt(text, square):
    result = ''
    size = len(square)
    if len(text) < size * size:
        text += 'x' * (size * size - len(text))
    indices = convertToIndices(square)
    result = ''.join([''.join([text[num] for num in row]) for row in indices])
    return result


def magicSquareDecrypt(text, square):
    result = ''
    size = len(square)
    indices = convertToIndices(square)
    index = 0
    for _ in range(size):
        for i in range(size):
            for j in range(size):
                if indices[i][j] == index:
                    result += text[i * size + j]
                    index += 1
    return result


def convertToIndices(matrix):
    ls = [val for row in matrix for val in row]
    sorted_indices = sorted(range(len(ls)), key=lambda k: ls[k])
    indexes = [0]*len(ls)
    for i, x in enumerate(sorted_indices):
        indexes[x] = i
    matrix = [[indexes[i*len(matrix) + j] for j in range(len(matrix))]
              for i in range(len(matrix))]
    return matrix


def getGridFromSquare(text, square):
    size = len(square)
    if len(text) < size * size:
        text += 'x' * (size * size - len(text))
    indices = convertToIndices(square)
    index = 0
    for row in indices:
        grid[index] = []
        for num in row:
            grid[index].append(text[num])
        index += 1
    return grid[0: size]


text = 'Isyuk Nikita'
square = [
    [7,  8, 14, 16],
    [6, 10, 11, 18],
    [1, 15, 12, 17],
    [3,  5, 13, 24],
]
encrypt = magicSquareEncrypt(text, square)
decrypt = magicSquareDecrypt(encrypt, square)
print(
    'Квадрат: \n' +
    '[{} ]'.format(' ]\n['.join(','.join(['{:3d}'.format(item) for item in row]) for row in square)) +
    '\n{}'.format(''.join(['-' for i in range(len(square) * 5)])) +
    '\n{}'.format('\n'.join([str(row) for row in getGridFromSquare(text, square)]) +
                  '\nИсходный текст:         {}\n'.format(text) +
                  'Зашифрованный текст:    {}\n'.format(encrypt) +
                  'Дешифрованный текст:    {}'.format(decrypt)))

Квадрат: 
[  7,  8, 14, 16 ]
[  6, 10, 11, 18 ]
[  1, 15, 12, 17 ]
[  3,  5, 13, 24 ]
--------------------
['k', ' ', 't', 'x']
['u', 'N', 'i', 'x']
['I', 'a', 'k', 'x']
['s', 'y', 'i', 'x']
Исходный текст:         Isyuk Nikita
Зашифрованный текст:    k txuNixIakxsyix
Дешифрованный текст:    Isyuk Nikita


### Двойная перестановка

In [227]:
def doublePermutationCipherEncrypt(text, colsOrder, rowsOrder, seed=45):
    result = ''
    random.seed(seed)
    cols = len(colsOrder)
    rows = len(rowsOrder)
    size = cols * rows
    text.replace('j', 'i')
    if len(text) % size != 0:
        text += 'j'
        for i in range(size - len(text)):
            text += chr(random.randint(ord('a'), ord('z')))
    grid = [text[i:i+cols] for i in range(0, len(text), cols)]
    grid = [[grid[i][colsOrder[j]]
             for j in range(cols)] for i in range(len(grid))]
    grid = [[grid[rowsOrder[i]][j]
             for j in range(cols)] for i in range(len(grid))]
    for i in range(cols):
        for j in range(rows):
            result += grid[j][i]
    return result


def doublePermutationCipherDecrypt(text, colsOrder, rowsOrder):
    cols = len(colsOrder)
    rows = len(rowsOrder)
    result = ''
    grid = [[text[i * cols + j] for i in range(cols)] for j in range(rows)]
    grid = [[grid[rowsOrder[i]][j]
             for j in range(cols)] for i in range(len(grid))]
    grid = [[grid[i][colsOrder[j]]
             for j in range(cols)] for i in range(len(grid))]

    result = ''.join([''.join(row) for row in grid])

    return result[0:result.index('j')]


def getGrid(text, cols, rows, seed=45):
    random.seed(seed)
    if len(text) % (cols * rows) != 0:
        text += 'j'
        for i in range(cols * rows - len(text)):
            text += chr(random.randint(ord('a'), ord('z')))
    grid = []
    for i in range(rows):
        grid.append([])
        for j in range(cols):
            grid[i].append(text[i * cols + j])
    return grid


text = "Isyuk Nikita"
colsOrder = [0, 2, 1, 3, 4]  # cols
rowsOrder = [2, 3, 0, 1, 4]  # rows
encrypt = doublePermutationCipherEncrypt(text, colsOrder, rowsOrder)
decrypt = doublePermutationCipherDecrypt(encrypt, colsOrder, rowsOrder)
grid = getGrid(text, len(colsOrder), len(rowsOrder))


def formatOutput(grid, colsOrder, rowsOrder):
    result = ''
    result += '  |   '
    result += '    '.join([str(item) for item in colsOrder])
    result += ' → Индексы столбцов при замене\n'
    result += '{}\n'.format(''.join(['-' for i in range(len(colsOrder) * 6)]))
    for i in range(len(grid)):
        result += f'{rowsOrder[i]} | '
        result += ''.join(str(grid[i]))
        result += '\n'
    result += '↳ Индексы строк при замене'
    return result


print('Матрица: (запись в матрицу →↓\n' +
      '          выписывание из матрицы ↓→)\n' +
      formatOutput(grid, colsOrder, rowsOrder) +
      '\n\nИсходный текст:         {}\n'.format(text) +
      'Зашифрованный текст:    {}\n'.format(encrypt) +
      'Дешифрованный текст:    {}'.format(decrypt))

Матрица: (запись в матрицу →↓
          выписывание из матрицы ↓→)
  |   0    2    1    3    4 → Индексы столбцов при замене
------------------------------
2 | ['I', 's', 'y', 'u', 'k']
3 | [' ', 'N', 'i', 'k', 'i']
0 | ['t', 'a', 'j', 'i', 'n']
1 | ['p', 'i', 'c', 'j', 'k']
4 | ['a', 'c', 'p', 'a', 'd']
↳ Индексы строк при замене

Исходный текст:         Isyuk Nikita
Зашифрованный текст:    tpI ajcyipaisNcijukankkid
Дешифрованный текст:    Isyuk Nikita
